In [ ]:
# Main imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from collections import deque
import math
%matplotlib inline
from importlib import reload
import utils2; reload(utils2)
from utils2 import *

In [ ]:
test_imgs_dir = "test_img"
test_imgs_paths = glob.glob(test_imgs_dir + "/*.jpg")
test_img_names = np.asarray(list(map(lambda img_path: img_path.split("/")[-1].split(".")[0], test_imgs_paths)))
undist_test_img_names = np.asarray(list(map(lambda img_name: "{0}{1}".format("undistorted_", img_name), test_img_names)))
test_imgs = np.asarray(list(map(lambda img_path: load_image(img_path), test_imgs_paths)))

In [ ]:
# FINAL FUNCTIONS
def threshold_img(img, channel, thres=(0, 255)):
    """
    Applies a threshold mask to the input image
    """
    img_ch = img[:,:,channel]
    if thres is None:
        return img_ch

    mask_ch = np.zeros_like(img_ch)
    mask_ch[ (thres[0] <= img_ch) & (thres[1] >= img_ch) ] = 1
    return mask_ch


def compute_hsv_white_red_binary(rgb_img):
    """
    Returns a binary thresholded image produced retaining only white and yellow elements on the picture
    The provided image should be in RGB format
    """
    hsv_img = to_hsv(rgb_img)

    # Red color thresholds
    red_hue_min = int(0 * 179)  # Lower bound for red hue
    red_hue_max = int(179)  # Upper bound for red hue (allowing some range)
    red_saturation_min = int(0.5 * 255)  # Lower bound for red saturation
    red_saturation_max = int(1.0 * 255)  # Upper bound for red saturation
    red_value_min = int(0.5 * 255)  # Lower bound for red value
    red_value_max = int(1 * 255)  # Upper bound for red value

    img_hsv_red_bin = np.zeros_like(hsv_img[:,:,0])
    img_hsv_red_bin[((hsv_img[:,:,0] >= red_hue_min) & (hsv_img[:,:,0] <= red_hue_max))
                    & ((hsv_img[:,:,1] >= red_saturation_min) & (hsv_img[:,:,1] <= red_saturation_max))
                    & ((hsv_img[:,:,2] >= red_value_min) & (hsv_img[:,:,2] <= red_value_max))] = 1

    # White color thresholds
    white_saturation_max = int(0.3 * 255)  # Adjusted saturation maximum for white
    white_value_min = int(0.9 * 255)  # Adjusted value minimum for white

    img_hsv_white_bin = np.zeros_like(hsv_img[:,:,0])
    img_hsv_white_bin[((hsv_img[:,:,1] <= white_saturation_max))
                      & ((hsv_img[:,:,2] >= white_value_min))] = 1

    # Combine both red and white binary images
    img_hls_white_red_bin = np.zeros_like(hsv_img[:,:,0])
    img_hls_white_red_bin[(img_hsv_red_bin == 1) | (img_hsv_white_bin == 1)] = 1

    return img_hls_white_red_bin



def abs_sobel(gray_img, x_dir=True, kernel_size=3, thres=(0, 255)):
    """
    Applies the sobel operator to a grayscale-like (i.e. single channel) image in either horizontal or vertical direction
    The function also computes the asbolute value of the resulting matrix and applies a binary threshold
    """
    sobel = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=kernel_size) if x_dir else cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=kernel_size)
    sobel_abs = np.absolute(sobel)
    sobel_scaled = np.uint8(255 * sobel / np.max(sobel_abs))
    gradient_mask = np.zeros_like(sobel_scaled)
    gradient_mask[(thres[0] <= sobel_scaled) & (sobel_scaled <= thres[1])] = 1
    return gradient_mask

def mag_sobel(gray_img, kernel_size=3, thres=(0, 255)):
    """
    Computes sobel matrix in both x and y directions, merges them by computing the magnitude in both directions
    and applies a threshold value to only set pixels within the specified range
    """
    sx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sy = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=kernel_size)

    sxy = np.sqrt(np.square(sx) + np.square(sy))
    scaled_sxy = np.uint8(255 * sxy / np.max(sxy))

    sxy_binary = np.zeros_like(scaled_sxy)
    sxy_binary[(scaled_sxy >= thres[0]) & (scaled_sxy <= thres[1])] = 1

    return sxy_binary
def dir_sobel(gray_img, kernel_size=3, thres=(0, np.pi/2)):
    """
    Computes sobel matrix in both x and y directions, gets their absolute values to find the direction of the gradient
    and applies a threshold value to only set pixels within the specified range
    """
    sx_abs = np.absolute(cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=kernel_size))
    sy_abs = np.absolute(cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=kernel_size))

    dir_sxy = np.arctan2(sx_abs, sy_abs)

    binary_output = np.zeros_like(dir_sxy)
    binary_output[(dir_sxy >= thres[0]) & (dir_sxy <= thres[1])] = 1

    return binary_output
def combined_sobels(sx_binary, sy_binary, sxy_magnitude_binary, gray_img, kernel_size=3, angle_thres=(0, np.pi/2)):
    sxy_direction_binary = dir_sobel(gray_img, kernel_size=kernel_size, thres=angle_thres)

    combined = np.zeros_like(sxy_direction_binary)
    # Sobel X returned the best output so we keep all of its results. We perform a binary and on all the other sobels
    combined[(sx_binary == 1) | ((sy_binary == 1) & (sxy_magnitude_binary == 1) & (sxy_direction_binary == 1))] = 1

    return combined
def compute_perspective_transform_matrices(src, dst):
    """
    Returns the tuple (M, M_inv) where M represents the matrix to use for perspective transform
    and M_inv is the matrix used to revert the transformed image back to the original one
    """
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)

    return (M, M_inv)

def perspective_transform(img, src, dst):
    """
    Applies a perspective
    """
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    return warped

In [ ]:
test_img_path = test_imgs_paths[0]

In [ ]:
test_img = load_image(test_img_path)
outside_test_img_path = test_imgs_paths[0]
outside_test_img = load_image(outside_test_img_path)
rgb_comp = np.asarray([[threshold_img(test_img, 0, thres=None), threshold_img(test_img, 1, thres=None), threshold_img(test_img, 2, thres=None)]])
rgb_lbs = np.asarray([["Red Channel", "Green Channel", "Blue Channel"]])
hls_test_img = to_hls(test_img)
hls_comp = np.asarray([[threshold_img(hls_test_img, 0, thres=None), threshold_img(hls_test_img, 1, thres=None), threshold_img(hls_test_img, 2, thres=None)]])
hls_lbs = np.asarray([["Hue Channel", "Lightness Channel", "Saturation Channel"]])
hsv_test_img = to_hsv(test_img)
hsv_comp = np.asarray([[threshold_img(hsv_test_img, 0, thres=None), threshold_img(hsv_test_img, 1, thres=None), threshold_img(hsv_test_img, 2, thres=None)]])
hsv_lbs = np.asarray([["Hue Channel", "Saturation Channel", "Value Channel"]])
lab_test_img = to_lab(test_img)
lab_comp = np.asarray([[threshold_img(lab_test_img, 0, thres=None), threshold_img(lab_test_img, 1, thres=None), threshold_img(lab_test_img, 2, thres=None)]])
lab_lbs = np.asarray([["Lightness Channel", "Green-Red (A) Channel", "Blue-Yellow (B) Channel"]])
color_spaces_comps = np.concatenate((rgb_comp, hls_comp, hsv_comp, lab_comp))
color_spaces_lbs = np.concatenate((rgb_lbs, hls_lbs, hsv_lbs, lab_lbs))

In [ ]:
white_red_hsv_img_bin = compute_hsv_white_red_binary(test_img)

# fig, ax = plt.subplots(1, 2, figsize=(10,7))
# ax[0].imshow(test_img)
# ax[0].axis("off")
# ax[0].set_title("Undistorted Image")

# ax[1].imshow(undistorted_yellow_white_hsv_img_bin, cmap='gray')
# ax[1].axis("off")
# ax[1].set_title("HSV Color Thresholded Image")

# plt.show()

# _________________ Params 1 _________________

# hls_test_img threshold_img(hls_test_img, 2, thres=None)
# hsv_test_img threshold_img(hsv_test_img, 1, thres=None)
# lab_test_img threshold_img(lab_test_img, 1, thres=None)
hls_test_img = to_hls(test_img)
hsv_test_img = to_hsv(test_img)
lab_test_img = to_lab(test_img)

hls_image = threshold_img(hls_test_img, 2, thres=None)
hsv_image = threshold_img(hsv_test_img, 1, thres=None)
lab_image = threshold_img(lab_test_img, 1, thres=None)

# threshold_image = hsv_image
# threshold_image = hls_image
threshold_image = lab_image

In [ ]:
# _________ Params 2 _____________
sobx_11x11_thres = np.asarray([[abs_sobel(threshold_image, kernel_size=11, thres=(20, 120)), abs_sobel(threshold_image, kernel_size=11, thres=(50, 150)), abs_sobel(threshold_image, kernel_size=11, thres=(80, 200))]])
sobx_15x15_thres = np.asarray([[abs_sobel(threshold_image, kernel_size=15, thres=(20, 120)), abs_sobel(threshold_image, kernel_size=15, thres=(50, 150)), abs_sobel(threshold_image, kernel_size=15, thres=(80, 200))]])


sobx_11x11_thres_lbs = np.asarray([["11x11 - Threshold (20,120)", "11x11 - Threshold (50,150)", "11x11 - Threshold (80,200)"]])
sobx_15x15_thres_lbs = np.asarray([["15x15 - Threshold (20,120)", "15x15 - Threshold (50,150)", "15x15 - Threshold (80,200)"]])
sobx_thres = np.concatenate(( sobx_11x11_thres, sobx_15x15_thres))
sobx_thres_lbs = np.concatenate((sobx_11x11_thres_lbs, sobx_15x15_thres_lbs))



sobx_best = abs_sobel(threshold_image, kernel_size=15, thres=(80, 200))
show_image_list(sobx_thres, sobx_thres_lbs, "Sobel (X Direction) Thresholds", cols=3, show_ticks=False)


In [ ]:
# ___________ Perams 3 _____________

soby_11x11_thres = np.asarray([[abs_sobel(threshold_image, x_dir=False, kernel_size=11, thres=(20, 120)), abs_sobel(threshold_image, x_dir=False, kernel_size=11, thres=(50, 150)), abs_sobel(threshold_image, x_dir=False, kernel_size=11, thres=(80, 200))]])
soby_15x15_thres = np.asarray([[abs_sobel(threshold_image, x_dir=False, kernel_size=15, thres=(20, 120)), abs_sobel(threshold_image, x_dir=False, kernel_size=15, thres=(50, 150)), abs_sobel(threshold_image, x_dir=False, kernel_size=15, thres=(80, 200))]])

soby_11x11_thres_lbs = np.asarray([["11x11 - Threshold (20,120)", "11x11 - Threshold (50,150)", "11x11 - Threshold (80,200)"]])
soby_15x15_thres_lbs = np.asarray([["15x15 - Threshold (20,120)", "15x15 - Threshold (50,150)", "15x15 - Threshold (80,200)"]])
soby_thres = np.concatenate((soby_11x11_thres, soby_15x15_thres))
soby_thres_lbs = np.concatenate((soby_11x11_thres_lbs, soby_15x15_thres_lbs))

soby_best = abs_sobel(threshold_image, x_dir=False, kernel_size=11, thres=(50, 150))
show_image_list(soby_thres, soby_thres_lbs, "Sobel (Y Direction) Thresholds", cols=3, show_ticks=False)

In [ ]:
# ________ perams 3 _________
sobxy_11x11_thres = np.asarray([[mag_sobel(threshold_image, kernel_size=11, thres=(20, 80)), mag_sobel(threshold_image, kernel_size=11, thres=(50, 150)), mag_sobel(threshold_image, kernel_size=11, thres=(80, 200))]])
sobxy_15x15_thres = np.asarray([[mag_sobel(threshold_image, kernel_size=15, thres=(20, 80)), mag_sobel(threshold_image, kernel_size=15, thres=(50, 150)), mag_sobel(threshold_image, kernel_size=15, thres=(80, 200))]])

sobxy_11x11_thres_lbs = np.asarray([["11x11 - Threshold (20,80)", "11x11 - Threshold (50,150)", "11x11 - Threshold (80,200)"]])
sobxy_15x15_thres_lbs = np.asarray([["15x15 - Threshold (20,80)", "15x15 - Threshold (50,150)", "15x15 - Threshold (80,200)"]])
sobxy_thres = np.concatenate((sobxy_11x11_thres, sobxy_15x15_thres))
sobxy_thres_lbs = np.concatenate((sobxy_11x11_thres_lbs, sobxy_15x15_thres_lbs))

sobxy_best = mag_sobel(threshold_image, kernel_size=15, thres=(80, 200))
show_image_list(sobxy_thres, sobxy_thres_lbs, "Sobel (XY Magnitude) Thresholds", cols=3, show_ticks=False)


In [ ]:
# ___________ Perams 4 ______________

sobxy_combined_dir_11x11_thres = np.asarray([[combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=11, angle_thres=(0, np.pi/4)),
                                            combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=11, angle_thres=(np.pi/4, np.pi/2)),
                                            combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=11, angle_thres=(np.pi/3, np.pi/2))
                                           ]])

sobxy_combined_dir_15x15_thres = np.asarray([[combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=15, angle_thres=(0, np.pi/4)),
                                            combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=15, angle_thres=(np.pi/4, np.pi/2)),
                                            combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=15, angle_thres=(np.pi/3, np.pi/2))
                                           ]])


sobxy_combined_dir_11x11_thres_lbs = np.asarray([["11x11 - Combined (0, pi/4)", "11x11 - Combined (pi/4, pi/2)", "11x11 - Combined (pi/3, pi/2)"]])
sobxy_combined_dir_15x15_thres_lbs = np.asarray([["15x15 - Combined (0, pi/4)", "15x15 - Combined (pi/4, pi/2)", "15x15 - Combined (pi/3, pi/2)"]])
sobxy_combined_dir_thres = np.concatenate((sobxy_combined_dir_11x11_thres, sobxy_combined_dir_15x15_thres))
sobxy_combined_dir_thres_lbs = np.concatenate((sobxy_combined_dir_11x11_thres_lbs, sobxy_combined_dir_15x15_thres_lbs))


sobel_combined_best = combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=15, angle_thres=(0, np.pi/4))
show_image_list(sobxy_combined_dir_thres, sobxy_combined_dir_thres_lbs, "Combined With Gradient Direction", cols=3, show_ticks=False)

In [ ]:
color_binary = np.dstack((np.zeros_like(sobel_combined_best), sobel_combined_best, white_red_hsv_img_bin)) * 255
color_binary = color_binary.astype(np.uint8)
combined_binary = np.zeros_like(white_red_hsv_img_bin)
combined_binary[(sobel_combined_best == 1) | (white_red_hsv_img_bin == 1)] = 1
combined_binaries = [[color_binary, combined_binary]]
combined_binaries_lbs = np.asarray([["Stacked Thresholds", "Combined Color And Gradient Thresholds"]])
show_image_list(combined_binaries, combined_binaries_lbs, "Color And Binary Combined Gradient And HLS (S) Thresholss", cols=2, fig_size=(17, 6), show_ticks=False)

In [ ]:

def get_combined_binary_thresholded_img(img):
    """
    Applies a combination of binary Sobel and color thresholding to an undistorted image
    Those binary images are then combined to produce the returned binary image
    """
    #Peramiter one
    # thresholded_img = threshold_img(img, 2, thres=None)
    threshold_image = threshold_img(img, 1, thres=None)

    #Peramiter two
    sobx_best = abs_sobel(threshold_image, kernel_size=15, thres=(80, 200))

    #Peramiter three
    soby_best = abs_sobel(threshold_image, x_dir=False, kernel_size=11, thres=(50, 150))

    #Peramiter four
    sobxy_best = mag_sobel(threshold_image, kernel_size=15, thres=(80, 200))

    #Peramiter five
    sobel_combined_best = combined_sobels(sobx_best, soby_best, sobxy_best, threshold_image, kernel_size=15, angle_thres=(0, np.pi/4))


    hsv_w_y_thres = compute_hsv_white_red_binary(img)
    combined_binary = np.zeros_like(hsv_w_y_thres)
    combined_binary[(sobel_combined_best == 1) | (hsv_w_y_thres == 1)] = 1


    # return sxy_combined_dir #DEBUG UNCOMMENT
    return combined_binary

In [ ]:
copy_combined = np.copy(test_imgs[1])
(bottom_px, right_px) = (copy_combined.shape[0] - 1, copy_combined.shape[1] - 1)
pts = np.array([[25,430],[375,345],[660,345], [right_px-20, 420]], np.int32)
cv2.polylines(copy_combined,[pts],True,(255,0,0), 10)
plt.axis('off')
plt.imshow(copy_combined)

In [ ]:
src_pts = pts.astype(np.float32)
dst_pts = np.array([[200, bottom_px], [200, 0], [right_px-200, 0], [right_px-200, bottom_px]], np.float32)
test_img_persp_tr = perspective_transform(test_imgs[3], src_pts, dst_pts)
plt.imshow(test_img_persp_tr)

In [ ]:
pts = np.array([[25,430],[375,345],[660,345], [right_px-20, 420]], np.int32)
src_pts = pts.astype(np.float32)
dst_pts = np.array([[200, bottom_px], [200, 0], [right_px-200, 0], [right_px-200, bottom_px]], np.float32)
per_img = perspective_transform(test_imgs[1], src_pts, dst_pts)
plt.imshow(per_img)


In [ ]:
test_imgs_pers_tr = np.asarray(list(map(lambda img: perspective_transform(img, src_pts, dst_pts), test_imgs)))
test_persp_img = np.copy(test_imgs_pers_tr[1])
dst = dst_pts.astype(np.int32)
cv2.polylines(test_persp_img,[dst],True,(255,0,0), 10)

fig, ax = plt.subplots(1, 2, figsize=(15,10))
ax[0].imshow(test_imgs_pers_tr[1])
ax[0].set_title("Perspecting Transform - Curved Lines")

ax[1].imshow(test_imgs[1])
ax[1].set_title("Perspective Transform - Straight Lines")

plt.show()

In [ ]:
test_undist_imgs_and_p_tr = np.asarray(list(zip(test_imgs, test_imgs_pers_tr)))
test_undist_imgs_and_p_tr_names = np.asarray(list(zip(undist_test_img_names, undist_test_img_names)))
show_image_list(test_undist_imgs_and_p_tr, test_undist_imgs_and_p_tr_names, "Undistorted and Birds View Image", fig_size=(15, 20))

In [ ]:
test_imgs_thresholded = np.asarray(list(map(lambda img: get_combined_binary_thresholded_img(img), test_imgs)))
test_imgs_and_thresholded = np.asarray(list(zip(test_imgs, test_imgs_thresholded)))
test_imgs_and_thresholded_names = np.asarray(list(zip(undist_test_img_names, undist_test_img_names)))
show_image_list(test_imgs_and_thresholded, test_imgs_and_thresholded_names, "Undistorted and Birds View Image", fig_size=(15, 20))

In [ ]:
test_imgs_combined_binary_thres = np.asarray(list(map(lambda img: get_combined_binary_thresholded_img(img), test_imgs)))
test_imgs_psp_tr = np.asarray(list(map(lambda img: perspective_transform(img, src_pts, dst_pts), test_imgs)))
test_imgs_combined_binary_psp_tr = np.asarray(list(map(lambda img: perspective_transform(img, src_pts, dst_pts), test_imgs_combined_binary_thres)))
test_imgs_combined_binary_and_psp_tr = np.asarray(list(zip(test_imgs_psp_tr[0],test_imgs_combined_binary_thres, test_imgs_combined_binary_psp_tr)))
test_imgs_combined_binary_and_psp_tr_names = np.asarray(list(zip(undist_test_img_names,undist_test_img_names, undist_test_img_names)))
show_image_list(test_imgs_combined_binary_and_psp_tr, test_imgs_combined_binary_and_psp_tr_names, "Combined Binary And Perspective Transform Images", cols=3, fig_size=(15, 15))

In [ ]:
img_example = test_imgs_combined_binary_and_psp_tr[3][2]
histogram = np.sum(img_example[img_example.shape[0]//2:,:], axis=0)
kernel_width = 30
smoothing_kernel = np.ones(kernel_width) / kernel_width

# Apply convolution to smooth the histogram
smoothed_histogram = np.convolve(histogram, smoothing_kernel, mode='same')
fig, ax = plt.subplots(1, 2, figsize=(15,4))
ax[0].imshow(img_example, cmap='gray')
ax[0].axis("off")
ax[0].set_title("Binary Thresholded Perspective Transform Image")

ax[1].plot(smoothed_histogram)
ax[1].set_title("Histogram Of Pixel Intensities (Image Bottom Half)")

plt.show()

In [ ]:
from collections import deque
import math

class LaneLineHistory:
    def __init__(self, queue_depth=2, test_points=[50, 300, 500, 700], poly_max_deviation_distance=150):
        self.lane_lines = create_queue(queue_depth)
        self.smoothed_poly = None
        self.test_points = test_points
        self.poly_max_deviation_distance = poly_max_deviation_distance

    def append(self, lane_line, force=False):
        if len(self.lane_lines) == 0 or force:
            self.lane_lines.append(lane_line)
            self.get_smoothed_polynomial()
            return True

        test_y_smooth = np.asarray(list(map(lambda x: self.smoothed_poly[0] * x**2 + self.smoothed_poly[1] * x + self.smoothed_poly[2], self.test_points)))
        test_y_new = np.asarray(list(map(lambda x: lane_line.polynomial_coeff[0] * x**2 + lane_line.polynomial_coeff[1] * x + lane_line.polynomial_coeff[2], self.test_points)))

        dist = np.absolute(test_y_smooth - test_y_new)

        #dist = np.absolute(self.smoothed_poly - lane_line.polynomial_coeff)
        #dist_max = np.absolute(self.smoothed_poly * self.poly_max_deviation_distance)
        max_dist = dist[np.argmax(dist)]

        if max_dist > self.poly_max_deviation_distance:
            print("**** MAX DISTANCE BREACHED ****")
            print("y_smooth={0} - y_new={1} - distance={2} - max-distance={3}".format(test_y_smooth, test_y_new, max_dist, self.poly_max_deviation_distance))
            return False

        self.lane_lines.append(lane_line)
        self.get_smoothed_polynomial()

        return True

    def get_smoothed_polynomial(self):
        all_coeffs = np.asarray(list(map(lambda lane_line: lane_line.polynomial_coeff, self.lane_lines)))
        self.smoothed_poly = np.mean(all_coeffs, axis=0)

        return self.smoothed_poly


def create_queue(length = 10):
    return deque(maxlen=length)
class LaneLine:
    def __init__(self):

        self.polynomial_coeff = None
        self.line_fit_x = None
        self.non_zero_x = []
        self.non_zero_y = []
        self.windows = []
class AdvancedLaneDetectorWithMemory:
    """
    The AdvancedLaneDetectorWithMemory is a class that can detect lines on the road
    """
    def __init__(self, psp_src, psp_dst, sliding_windows_per_line,
                 sliding_window_half_width, sliding_window_recenter_thres,
                 small_img_size=(256, 144), small_img_x_offset=20, small_img_y_offset=10,
                 img_dimensions=(720, 1280), lane_width_px=800,
                 lane_center_px_psp=600, real_world_lane_size_meters=(32, 3.7)):
        (self.M_psp, self.M_inv_psp) = compute_perspective_transform_matrices(psp_src, psp_dst)

        self.sliding_windows_per_line = sliding_windows_per_line
        self.sliding_window_half_width = sliding_window_half_width
        self.sliding_window_recenter_thres = sliding_window_recenter_thres

        self.small_img_size = small_img_size
        self.small_img_x_offset = small_img_x_offset
        self.small_img_y_offset = small_img_y_offset

        self.img_dimensions = img_dimensions
        self.lane_width_px = lane_width_px
        self.lane_center_px_psp = lane_center_px_psp
        self.real_world_lane_size_meters = real_world_lane_size_meters

        # We can pre-compute some data here
        self.ym_per_px = self.real_world_lane_size_meters[0] / self.img_dimensions[0]
        self.xm_per_px = self.real_world_lane_size_meters[1] / self.lane_width_px
        self.ploty = np.linspace(0, self.img_dimensions[0] - 1, self.img_dimensions[0])

        self.previous_left_lane_line = None
        self.previous_right_lane_line = None

        self.previous_left_lane_lines = LaneLineHistory()
        self.previous_right_lane_lines = LaneLineHistory()

        self.total_img_count = 0


    def process_image(self, img):
        """
        Attempts to find lane lines on the given image and returns an image with lane area colored in green
        as well as small intermediate images overlaid on top to understand how the algorithm is performing
        """
        # First step - undistort the image using the instance's object and image points


        # Produce binary thresholded image from color and gradients
        thres_img = get_combined_binary_thresholded_img(img)


        # Create the undistorted and binary perspective transforms
        img_size = (img.shape[1], img.shape[0])
        undist_img_psp = cv2.warpPerspective(img, self.M_psp, img_size, flags=cv2.INTER_LINEAR)
        thres_img_psp = cv2.warpPerspective(thres_img, self.M_psp, img_size, flags=cv2.INTER_LINEAR)

        ll, rl = self.compute_lane_lines(thres_img_psp)
        lcr, rcr, lco = self.compute_lane_curvature(ll, rl)

        drawn_lines = self.draw_lane_lines(thres_img_psp, ll, rl)

        drawn_lines_regions = self.draw_lane_lines_regions(thres_img_psp, ll, rl)

        drawn_lane_area = self.draw_lane_area(thres_img_psp, img, ll, rl)

        drawn_hotspots = self.draw_lines_hotspots(thres_img_psp, ll, rl)

        combined_lane_img = self.combine_images(drawn_lane_area, drawn_lines, drawn_lines_regions, drawn_hotspots, undist_img_psp)

        final_img = self.draw_lane_curvature_text(combined_lane_img, lcr, rcr, lco)

        self.total_img_count += 1
        self.previous_left_lane_line = ll
        self.previous_right_lane_line = rl
        print(lcr, rcr, lco)
        if -0.1 < lco and lco < 0.1:
          print('around center')
        elif lco < 0:
          print('move right by :', lco,'m')
        elif lco > 0:
          print('move left by :', lco,'m')
        # return undist_img_psp
        return final_img

    def draw_lane_curvature_text(self, img, left_curvature_meters, right_curvature_meters, center_offset_meters):
        """
        Returns an image with curvature information inscribed
        """

        offset_y = self.small_img_size[1] * 1 + self.small_img_y_offset * 5
        offset_x = self.small_img_x_offset

        template = "{0:17}{1:17}{2:17}"
        txt_header = template.format("Left Curvature", "Right Curvature", "Center Alignment")
        print(txt_header)
        txt_values = template.format("{:.4f}m".format(left_curvature_meters),
                                     "{:.4f}m".format(right_curvature_meters),
                                     "{:.4f}m Right".format(center_offset_meters))
        if center_offset_meters < 0.0:
            txt_values = template.format("{:.4f}m".format(left_curvature_meters),
                                     "{:.4f}m".format(right_curvature_meters),
                                     "{:.4f}m Left".format(math.fabs(center_offset_meters)))


        print(txt_values)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, txt_header, (offset_x, offset_y), font, 1, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(img, txt_values, (offset_x, offset_y + self.small_img_y_offset * 5), font, 1, (255,255,255), 2, cv2.LINE_AA)

        return img

    def combine_images(self, lane_area_img, lines_img, lines_regions_img, lane_hotspots_img, psp_color_img):
        """
        Returns a new image made up of the lane area image, and the remaining lane images are overlaid as
        small images in a row at the top of the the new image
        """
        small_lines = cv2.resize(lines_img, self.small_img_size)
        small_region = cv2.resize(lines_regions_img, self.small_img_size)
        small_hotspots = cv2.resize(lane_hotspots_img, self.small_img_size)
        small_color_psp = cv2.resize(psp_color_img, self.small_img_size)

        lane_area_img[self.small_img_y_offset: self.small_img_y_offset + self.small_img_size[1], self.small_img_x_offset: self.small_img_x_offset + self.small_img_size[0]] = small_lines

        start_offset_y = self.small_img_y_offset
        start_offset_x = 2 * self.small_img_x_offset + self.small_img_size[0]
        lane_area_img[start_offset_y: start_offset_y + self.small_img_size[1], start_offset_x: start_offset_x + self.small_img_size[0]] = small_region

        start_offset_y = self.small_img_y_offset
        start_offset_x = 3 * self.small_img_x_offset + 2 * self.small_img_size[0]
        lane_area_img[start_offset_y: start_offset_y + self.small_img_size[1], start_offset_x: start_offset_x + self.small_img_size[0]] = small_hotspots

        start_offset_y = self.small_img_y_offset
        start_offset_x = 4 * self.small_img_x_offset + 3 * self.small_img_size[0]
        lane_area_img[start_offset_y: start_offset_y + self.small_img_size[1], start_offset_x: start_offset_x + self.small_img_size[0]] = small_color_psp


        return lane_area_img


    def draw_lane_area(self, warped_img, img, left_line, right_line):
        """
        Returns an image where the inside of the lane has been colored in bright green
        """
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped_img).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        ploty = np.linspace(0, warped_img.shape[0] - 1, warped_img.shape[0])
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_line.line_fit_x, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.line_fit_x, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, self.M_inv_psp, (img.shape[1], img.shape[0]))
        # Combine the result with the original image
        result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)

        return result


    def draw_lane_lines(self, warped_img, left_line, right_line):
        """
        Returns an image where the computed lane lines have been drawn on top of the original warped binary image
        """
        # Create an output image with 3 colors (RGB) from the binary warped image to draw on and  visualize the result
        out_img = np.dstack((warped_img, warped_img, warped_img))*255

        # Now draw the lines
        ploty = np.linspace(0, warped_img.shape[0] - 1, warped_img.shape[0])
        pts_left = np.dstack((left_line.line_fit_x, ploty)).astype(np.int32)
        pts_right = np.dstack((right_line.line_fit_x, ploty)).astype(np.int32)

        cv2.polylines(out_img, pts_left, False,  (255, 140,0), 5)
        cv2.polylines(out_img, pts_right, False, (255, 140,0), 5)

        for low_pt, high_pt in left_line.windows:
            cv2.rectangle(out_img, low_pt, high_pt, (0, 255, 0), 3)

        for low_pt, high_pt in right_line.windows:
            cv2.rectangle(out_img, low_pt, high_pt, (0, 255, 0), 3)

        return out_img

    def draw_lane_lines_regions(self, warped_img, left_line, right_line):
        """
        Returns an image where the computed left and right lane areas have been drawn on top of the original warped binary image
        """
        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        margin = self.sliding_window_half_width
        ploty = np.linspace(0, warped_img.shape[0] - 1, warped_img.shape[0])

        left_line_window1 = np.array([np.transpose(np.vstack([left_line.line_fit_x - margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_line.line_fit_x + margin,
                                      ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))

        right_line_window1 = np.array([np.transpose(np.vstack([right_line.line_fit_x - margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_line.line_fit_x + margin,
                                      ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Create RGB image from binary warped image
        region_img = np.dstack((warped_img, warped_img, warped_img)) * 255

        # Draw the lane onto the warped blank image
        cv2.fillPoly(region_img, np.int_([left_line_pts]), (0, 255, 0))
        cv2.fillPoly(region_img, np.int_([right_line_pts]), (0, 255, 0))

        return region_img


    def draw_lines_hotspots(self, warped_img, left_line, right_line):
        """
        Returns a RGB image where the portions of the lane lines that were
        identified by our pipeline are colored in yellow (left) and blue (right)
        """
        out_img = np.dstack((warped_img, warped_img, warped_img))*255

        out_img[left_line.non_zero_y, left_line.non_zero_x] = [255, 255, 0]
        out_img[right_line.non_zero_y, right_line.non_zero_x] = [0, 0, 255]

        return out_img

    def compute_lane_curvature(self, left_line, right_line):
        """
        Returns the triple (left_curvature, right_curvature, lane_center_offset), which are all in meters
        """
        ploty = self.ploty
        y_eval = np.max(ploty)
        # Define conversions in x and y from pixels space to meters

        leftx = left_line.line_fit_x
        rightx = right_line.line_fit_x

        # Fit new polynomials: find x for y in real-world space
        print("Shape of ploty:", ploty.shape)
        print("Shape of leftx:", leftx.shape)
        print("Shape of rightx:", rightx.shape)
        left_fit_cr = np.polyfit(ploty * self.ym_per_px, leftx * self.xm_per_px, 2)
        right_fit_cr = np.polyfit(ploty * self.ym_per_px, rightx * self.xm_per_px, 2)

        # Now calculate the radii of the curvature
        left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * self.ym_per_px + left_fit_cr[1])**2)**1.5) / np.absolute(2 * left_fit_cr[0])
        right_curverad = ((1 + (2 *right_fit_cr[0] * y_eval * self.ym_per_px + right_fit_cr[1])**2)**1.5) / np.absolute(2 * right_fit_cr[0])

        # Use our computed polynomial to determine the car's center position in image space, then
        left_fit = left_line.polynomial_coeff
        right_fit = right_line.polynomial_coeff

        center_offset_img_space = (((left_fit[0] * y_eval**2 + left_fit[1] * y_eval + left_fit[2]) +
                   (right_fit[0] * y_eval**2 + right_fit[1] * y_eval + right_fit[2])) / 2) - self.lane_center_px_psp
        center_offset_real_world_m = center_offset_img_space * self.xm_per_px

        # Now our radius of curvature is in meters
        return left_curverad, right_curverad, center_offset_real_world_m



    def compute_lane_lines(self, warped_img):
        """
        Returns the tuple (left_lane_line, right_lane_line) which represents respectively the LaneLine instances for
        the computed left and right lanes, for the supplied binary warped image
        """

        # Take a histogram of the bottom half of the image, summing pixel values column wise
        histogram = np.sum(warped_img[warped_img.shape[0]//2:,:], axis=0)

        # Smooth the histogram
        kernel_width = 5
        smoothing_kernel = np.ones(kernel_width) / kernel_width

        # Apply convolution to smooth the histogram
        smoothed_histogram = np.convolve(histogram, smoothing_kernel, mode='same')

        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(smoothed_histogram.shape[0]//2)
        leftx_base = np.argmax(smoothed_histogram[:midpoint])
        rightx_base = np.argmax(smoothed_histogram[midpoint:]) + midpoint # don't forget to offset by midpoint!


        # Set height of windows
        window_height = np.int(warped_img.shape[0]//self.sliding_windows_per_line)
        # Identify the x and y positions of all nonzero pixels in the image
        # NOTE: nonzero returns a tuple of arrays in y and x directions
        nonzero = warped_img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        total_non_zeros = len(nonzeroy)
        non_zero_found_pct = 0.0

        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base


        # Set the width of the windows +/- margin
        margin = self.sliding_window_half_width
        # Set minimum number of pixels found to recenter window
        minpix = self.sliding_window_recenter_thres
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Our lane line objects we store the result of this computation
        left_line = LaneLine()
        right_line = LaneLine()

        if self.previous_left_lane_line is not None and self.previous_right_lane_line is not None:
            # We have already computed the lane lines polynomials from a previous image
            left_lane_inds = ((nonzerox > (self.previous_left_lane_line.polynomial_coeff[0] * (nonzeroy**2)
                                           + self.previous_left_lane_line.polynomial_coeff[1] * nonzeroy
                                           + self.previous_left_lane_line.polynomial_coeff[2] - margin))
                              & (nonzerox < (self.previous_left_lane_line.polynomial_coeff[0] * (nonzeroy**2)
                                            + self.previous_left_lane_line.polynomial_coeff[1] * nonzeroy
                                            + self.previous_left_lane_line.polynomial_coeff[2] + margin)))

            right_lane_inds = ((nonzerox > (self.previous_right_lane_line.polynomial_coeff[0] * (nonzeroy**2)
                                           + self.previous_right_lane_line.polynomial_coeff[1] * nonzeroy
                                           + self.previous_right_lane_line.polynomial_coeff[2] - margin))
                              & (nonzerox < (self.previous_right_lane_line.polynomial_coeff[0] * (nonzeroy**2)
                                            + self.previous_right_lane_line.polynomial_coeff[1] * nonzeroy
                                            + self.previous_right_lane_line.polynomial_coeff[2] + margin)))

            non_zero_found_left = np.sum(left_lane_inds)
            non_zero_found_right = np.sum(right_lane_inds)
            non_zero_found_pct = (non_zero_found_left + non_zero_found_right) / total_non_zeros

            print("[Previous lane] Found pct={0}".format(non_zero_found_pct))

        if non_zero_found_pct < 0.85:
            print("Non zeros found below thresholds, begining sliding window - pct={0}".format(non_zero_found_pct))
            left_lane_inds = []
            right_lane_inds = []

            # Step through the windows one by one
            for window in range(self.sliding_windows_per_line):
                # Identify window boundaries in x and y (and right and left)
                # We are moving our windows from the bottom to the top of the screen (highest to lowest y value)
                win_y_low = warped_img.shape[0] - (window + 1)* window_height
                win_y_high = warped_img.shape[0] - window * window_height

                # Defining our window's coverage in the horizontal (i.e. x) direction
                # Notice that the window's width is twice the margin
                win_xleft_low = leftx_current - margin
                win_xleft_high = leftx_current + margin
                win_xright_low = rightx_current - margin
                win_xright_high = rightx_current + margin

                left_line.windows.append([(win_xleft_low,win_y_low),(win_xleft_high,win_y_high)])
                right_line.windows.append([(win_xright_low,win_y_low),(win_xright_high,win_y_high)])

                # Super crytic and hard to understand...
                # Basically nonzerox and nonzeroy have the same size and any nonzero pixel is identified by
                # (nonzeroy[i],nonzerox[i]), therefore we just return the i indices within the window that are nonzero
                # and can then index into nonzeroy and nonzerox to find the ACTUAL pixel coordinates that are not zero
                good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
                good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

                # Append these indices to the lists
                left_lane_inds.append(good_left_inds)
                right_lane_inds.append(good_right_inds)

                # If you found > minpix pixels, recenter next window on their mean position
                if len(good_left_inds) > minpix:
                    leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
                if len(good_right_inds) > minpix:
                    rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

            # Concatenate the arrays of indices since we now have a list of multiple arrays (e.g. ([1,3,6],[8,5,2]))
            # We want to create a single array with elements from all those lists (e.g. [1,3,6,8,5,2])
            # These are the indices that are non zero in our sliding windows
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)

            non_zero_found_left = np.sum(left_lane_inds)
            non_zero_found_right = np.sum(right_lane_inds)
            non_zero_found_pct = (non_zero_found_left + non_zero_found_right) / total_non_zeros

            print("[Sliding windows] Found pct={0}".format(non_zero_found_pct))


        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        #print("[LEFT] Number of hot pixels={0}".format(len(leftx)))
        #print("[RIGHT] Number of hot pixels={0}".format(len(rightx)))
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        #print("Poly left {0}".format(left_fit))
        #print("Poly right {0}".format(right_fit))
        left_line.polynomial_coeff = left_fit
        right_line.polynomial_coeff = right_fit

        if not self.previous_left_lane_lines.append(left_line):
            left_fit = self.previous_left_lane_lines.get_smoothed_polynomial()
            left_line.polynomial_coeff = left_fit
            self.previous_left_lane_lines.append(left_line, force=True)
            print("**** REVISED Poly left {0}".format(left_fit))
        #else:
            #left_fit = self.previous_left_lane_lines.get_smoothed_polynomial()
            #left_line.polynomial_coeff = left_fit


        if not self.previous_right_lane_lines.append(right_line):
            right_fit = self.previous_right_lane_lines.get_smoothed_polynomial()
            right_line.polynomial_coeff = right_fit
            self.previous_right_lane_lines.append(right_line, force=True)
            print("**** REVISED Poly right {0}".format(right_fit))
        #else:
            #right_fit = self.previous_right_lane_lines.get_smoothed_polynomial()
            #right_line.polynomial_coeff = right_fit



        # Generate x and y values for plotting
        ploty = np.linspace(0, warped_img.shape[0] - 1, warped_img.shape[0] )
        left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]


        left_line.polynomial_coeff = left_fit
        left_line.line_fit_x = left_fitx
        left_line.non_zero_x = leftx
        left_line.non_zero_y = lefty

        right_line.polynomial_coeff = right_fit
        right_line.line_fit_x = right_fitx
        right_line.non_zero_x = rightx
        right_line.non_zero_y = righty


        return (left_line, right_line)

In [ ]:
test_img = test_imgs[4]

# Scale perspective to size of test inage
ctrlx = 1042
ctrly = 590
x_dim = test_img.shape[1]
y_dim = test_img.shape[0]
bottom_px = y_dim - 1
right_px = x_dim - 1
pts = np.array([[25*x_dim/ctrlx,430*y_dim/ctrly],[375*x_dim/ctrlx,345*y_dim/ctrly],[660*x_dim/ctrlx,345*y_dim/ctrly], [(right_px-20)*x_dim/ctrlx, 420*y_dim/ctrly]], np.int32)
src_pts = pts.astype(np.float32)
dst_pts = np.array([[200, bottom_px], [200, 0], [right_px-200, 0], [right_px-200, bottom_px]], np.float32)
# ^ May need to alter dst_pts

ld = AdvancedLaneDetectorWithMemory(src_pts, dst_pts, 20, 200, 50, img_dimensions=(y_dim,x_dim),real_world_lane_size_meters=(2.1,1.05),small_img_size=(128,72))
proc_img = ld.process_image(test_img)
plt.imshow(proc_img)